In [1]:
merge_file = r'\\new-monster\T\freely_moving_ephys\ephys_recordings\101520\G6H28P6LT\fm1_hf1_ephys\101520_fm1_hf1_merge.mat'


In [37]:
# package imports
import pandas as pd
import numpy as np
import xarray as xr
import os

# module imports
#from util.read_data import open_time
from scipy.io import loadmat

merge_info = loadmat(merge_file)
fileList = merge_info['fileList']
pathList = merge_info['pathList']
nSamps = merge_info['nSamps']

phy_path = os.path.split(merge_file)
allSpikeT = np.load(os.path.join(phy_path[0],'spike_times.npy'))
clust = np.load(os.path.join(phy_path[0],'spike_clusters.npy'))
templates = np.load(os.path.join(phy_path[0],'spike_templates.npy'))

boundaries = np.concatenate((np.array([0]),np.cumsum(nSamps)))

In [55]:
for s in range(len(nSamps)+1):
    print(s) 
    use = (allSpikeT >= boundaries[s]) & (allSpikeT<boundaries[s+1] )
    theseSpikes = allSpikeT[use]
    theseClust = clust[use]
    print(len(theseSpikes))

0
77104
1
29895


In [52]:
np.arange(len(nSamps))

array([0])

In [54]:
for s in range(2):
    print(s)

0
1


In [ ]:

allSpikeT = np.load(spike_times_path)
    allSpikeT = allSpikeT/config['ephys_sample_rate']  # should be a lookup table with timestamps
    duration = np.max(allSpikeT)
    ephys_data = pd.read_csv(cluster_info_path,sep = '\t',index_col=0)
    clust = np.load(spike_clusters_path)
    ephys_data['spikeT'] = np.nan
    ephys_data['spikeT'] = ephys_data['spikeT'].astype(object)
    # get spiketimes for each cluster
    for c in np.unique(clust):
        ephys_data.at[c,'spikeT'] =allSpikeT[clust==c].flatten()
    # get waveform templates
    templates = np.load(templates_path)
    ephys_data['waveform'] = np.nan
    ephys_data['waveform'] = ephys_data['spikeT'].astype(object)
    for i, ind in enumerate(ephys_data.index):
        ephys_data.at[ind,'waveform'] = templates[ind,21:,ephys_data.at[ind,'ch']]
    # first timepoint
    ephys_data['t0'] = open_time(ephys_time_path)[0]